In [1]:
import openai
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
    retry_if_exception_type
)  # for exponential backoff


In [ ]:
openai.organization = "" ## organization id
openai.api_key = ""  ## open api key
# Read Answers from CSV file
df = pd.read_csv(r'', header=None)
answers = df.iloc[1:, 1].tolist()

In [ ]:
Explanation_scheme_step_and_score_C1="""
As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Re-evaluate your thought process step by step to show how it reflects the grading scheme. Provide your grade as a single integer value.
The text you are grading is an answer to a question. Your evaluation should be based on the combined criteria of problem comprehension and explanation quality.


Please consider the following aspects while grading:
- Problem Comprehension: Assess how effectively the text demonstrates understanding of the problem and its relevance. Focus on the introduction and information presented throughout the text.
- Explanation Quality: Evaluate the informativeness of the explanation provided. Ensure that the text offers sufficient information to clarify the problem concisely.


Please use the following grading scheme:
5 points: The text demonstrates a very effective understanding of the problem, is closely related to the issue, and provides necessary information.
4 points: The text demonstrates a generally effective understanding of the problem, is closely related to the issue, and includes mostly necessary information.
3 points: The text shows an understanding of the problem related to the issue but lacks some information.
2 points: The text's understanding of the problem contains information that is not closely related to the issue.
1 point: The text provides no explanation of the problem or displays a very weak understanding.

Now, carefully review the following question and answer, and assign a grade accordingly:
"""

In [ ]:
@retry(
    retry=retry_if_exception_type((openai.error.APIError, openai.error.APIConnectionError, openai.error.RateLimitError, openai.error.ServiceUnavailableError, openai.error.Timeout)), 
    wait=wait_random_exponential(multiplier=1, max=60), 
    stop=stop_after_attempt(10)
)
def get_grade_gpt4(prompt_type, QA):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": prompt_type},
            {"role": "user", "content": QA}
        ],
        temperature = 0.1
    )
    response_text = response['choices'][0]['message']['content'].strip()
    return response_text

def get_grade_gpt3(prompt_type, QA):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": prompt_type},
            {"role": "user", "content": QA}
        ],
        temperature = 0.1 
        
    )
    response_text = response['choices'][0]['message']['content'].strip()
    return response_text

In [ ]:
# GPT4
# Process prompts and get responses
C2_responses_gpt4= []
for answer in answers:
    response_text = get_grade_gpt3(Explanation_scheme_step_and_score_C2, answer)
    C2_responses_gpt4.append(response_text)


# Create a new DataFrame with the responses
C2_responses_gpt4_df = pd.DataFrame({'C2_responses_gpt4_gpt3.5': C2_responses_gpt4})

# Load existing Excel file
file_path = r''
writer = pd.ExcelWriter(file_path, engine='openpyxl')

# Write the new DataFrame to the Excel file
C2_responses_gpt4_df.to_excel(writer, index=False, header=['C2_responses_gpt4'])
C2_responses_gpt4_df.to_csv(r"")
# Save and close the Excel file
writer.close()